In [2]:
#!/usr/bin/python
# -*- coding: UTF-8 -*-
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.regression import LinearRegressionWithSGD, LinearRegressionModel
from pyspark.mllib.regression import LassoWithSGD, LassoModel
from pyspark.mllib.regression import RidgeRegressionWithSGD, RidgeRegressionModel

from pyspark import SparkConf, SparkContext


In [3]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.replace(',', ' ').split(' ')]
    return LabeledPoint(values[0], values[1:])


In [4]:
data = sc.textFile("/root/spark-2.3.1-bin-hadoop2.6/data/mllib/ridge-data/lpsa.data")
parsedData = data.map(parsePoint)

In [5]:
model = LinearRegressionWithSGD.train(parsedData, iterations=100, step=0.00000001) # Linear least squares

D:\spark-2.0.2-bin-hadoop2.6\python\pyspark\mllib\regression.py:281: UserWarning: Deprecated in 2.0.0. Use ml.regression.LinearRegression.
  warnings.warn("Deprecated in 2.0.0. Use ml.regression.LinearRegression.")


In [6]:
model

(weights=[1.44020946095e-08,1.06866747363e-08,9.60897349531e-09,4.5534099838e-09,1.22214965608e-08,8.91077340698e-09,5.5962085584e-09,1.22556991288e-08], intercept=0.0)

In [7]:
valuesAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))

In [8]:
MSE = valuesAndPreds \
      .map(lambda vp: (vp[0] - vp[1])**2) \
      .reduce(lambda x, y: x + y) / valuesAndPreds.count()
print("Mean Squared Error = " + str(MSE))

Mean Squared Error = 7.4510328101
